# Introduction to Data Science – Homework 6
*COMP 5360 / MATH 4100, University of Utah, http://datasciencecourse.net/*

Due: Friday, February 23, 11:59pm.

In Part 1 of this homework you will scrape github repositories and organize the information in a Pandas dataframe. In Part 2, you will use linear regression to gain meaningful insights. 

## Your Data
First Name: Brooke
<br>
Last Name: Stanislawski
<br>
E-mail: brooke.stanislawski@utah.edu
<br>
UID: u1142223
<br>

In [1]:
# imports and setup 
from bs4 import BeautifulSoup
# you can use either of these libraries to get html from a website
import requests
import urllib.request


import pandas as pd
import scipy as sc
import numpy as np

import statsmodels.formula.api as sm

import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6) 

### 1. Scrape Github Repository List and Repository Information using BeautifulSoup
In this part you will explore over 2 million Github repositories. You are going to scrape [this repository list](https://github.com/search?o=desc&q=stars%3A%3E1&s=stars&type=Repositories).

### 1.1. Check whether you are permitted to scrape the data
Before you start to scrape any website you should go through the terms and services or similar pages of the website. Almost all websites post conditions to use their data. Check the terms of [https://github.com/](https://github.com/) to see whether the site permits you to scrape their data or not. Are you sure you are allowed to scrape?

**Your Interpretation:**
TODO

### Task 1.2 Download the website

Download the **first ten** pages of the list of highly starred repositories here: 

https://github.com/search?o=desc&q=stars%3A%3E1&s=stars&type=Repositories

To avoid sending too many requests to the server, make sure that your querying code is in a sperate cell from anything else, so that you don't have to re-run it needlessly. 

**Warning:** while we haven't experienced any problems, exepct that github will temporarily block access if you scrape their website relentlessly. Make sure to be responsible. Also, don't do this last minute – if you're blocked for e.g., 24 hours that could impact your timeline.

Now read the html text in python and create a soup object.

In [22]:
## Your code goes here

# download the fist ten pages

#for i in np.arange(1,11):
for i in np.arange(1,11):

    url = "https://github.com/search?o=desc&p="+str(i)+"&q=stars%3A%3E1&s=stars&type=Repositories"

    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')

    #A simple solution is to just use ‘response.text’ to get html returned by requests.get(url) directly. This doesn't work on Windows.

    # html = requests.get(url)
    # html = html.text

    # # save the file
    filename = "repos"+str(i)+".html"
    with open(filename, 'w') as new_file:
        new_file.write(str(html.replace('\n','').encode('utf-8')))
    
    # create one massive soup
    if i == 1:
        git_soup = BeautifulSoup(html, 'html.parser') # initialize bowl_soup w/ first page
    else:
        cup_soup = BeautifulSoup(html, 'html.parser') # soup for each page
        for element in cup_soup:
            git_soup.body.append(element) # massive soup that contains soups of all pages
    
        

In [23]:
# Creating the soup
# soup was created in for loop above
# git_soup = BeautifulSoup(html, 'html.parser')
git_soup


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://assets-cdn.github.com" rel="dns-prefetch"/>
<link href="https://avatars0.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars1.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars2.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars3.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/frameworks-7a12427f1445.css" media="all" rel="stylesheet">
<link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/github-fca0cb55603c.css" media="all" rel="stylesheet">
<link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/site-e1e1bc98a53e.css" media="all" rel="stylesheet">
<meta content="width=device-width" name="vi

### 1.3. Extract Data

Extract the foolowing data for each repository, and create a Pandas Dataframe with a row for each repository and a column for each of these datums. 

1. Name of the repository
+ URL of the repository 
+ Number of Contributors 
+ Programming Language Used 
+ Number of stars 
+ Number of issues
+ Number of files 
+ Length of readme file.

Note that you will frist have to extract links from the soup that you just scraped earlier, and then download the repository pages to retreive most of the data. 

Refer to lecture 12 for details on how to do this. Make sure to use the web inspector to identify the relevant structures. 

Save the dataframe you created to a new file `project_info.csv` and include this in your submission.

In [62]:
## Your code goes here

name_list = [] #
url_list = [] #
cont_list = [] #
lang_list = [] # 
stars_list = []
issues_list = []
forks_list = []
rmlength_list = []

repos = git_soup.find_all("div",class_="col-8 pr-3")
repos_lang = git_soup.find_all("div",class_="d-table-cell col-2 text-gray pt-2")
url = "https://github.com"

for i in range(20):
    name_list.append(repos[i].find("a",class_="v-align-middle").text)
    print(repos[i].find("a",class_="v-align-middle").text)
    url_tail = repos[i].find("a").get("href")
    repo_url = url + url_tail
    url_list.append(repo_url)
    with urllib.request.urlopen(repo_url) as response:
        repo_html = response.read()
        repo_html = repo_html.decode('utf-8')
    # if i%5==0:
    #     time.sleep(5) #sleeps after every 5 request
    #     # adjust frequency or time for sleeping if have issues
    with open('repo.html', 'w') as new_file:
        new_file.write(str(repo_html.replace('\n','').encode('utf-8')))
    repo_soup = BeautifulSoup(repo_html, 'html.parser')
    cont_list.append(repo_soup.find("a", href= lambda x : x and "contributors" in x).text.replace('\n','').strip('contributors').strip())
    lang_list.append(repos_lang[i].text.strip())
    stars_list.append(repo_soup.find("a",class_="social-count js-social-count").text.strip())
    issues_list.append(repo_soup.find("span",class_="Counter").text)
    forks_list.append(repo_soup.find("a", href= lambda x : x and "network" in x).text.strip())
    #print(len(repo_soup.find("article",class_="markdown-body entry-content").text.replace("\n","")))
    #rmlength_list.append(len(repo_soup.find("article",class_="markdown-body entry-content").text.replace("\n","")))
    #rmlength_list.append(len(repo_soup.find(id="readme").text.replace("\n","")))
    #rmlength_list.append(len(repo_soup.find(id="readme").get_text))
    rmlength_list.append(len(repo_soup.find("div",id="readme").text.replace("\n","")))
    print(len(repo_soup.find("div",id="readme").text.replace("\n","")))

    
print('names:',name_list,'\n')
print('URLs:',url_list,'\n')
print('cont_list:',cont_list,'\n')
print('lang_list:',lang_list,'\n')
print('# of stars:',stars_list,'\n')
print('# of issues:',issues_list,'\n')
print('# of forks:',forks_list,'\n')
print('RM lengths:',rmlength_list,'\n')


robbyrussell/oh-my-zsh
7298
github/gitignore
3268
facebook/react-native
4030
jwasham/coding-interview-university
66821
angular/angular.js
5194
electron/electron
2744
torvalds/linux
720
FortAwesome/Font-Awesome
3595
daneden/animate.css
6045
jquery/jquery
10896
freeCodeCamp/freeCodeCamp
2758
twbs/bootstrap
4908
EbookFoundation/free-programming-books
1512
tensorflow/tensorflow
2981
facebook/react
3263
vuejs/vue
2604
sindresorhus/awesome
12280
getify/You-Dont-Know-JS
2628
d3/d3
1499
airbnb/javascript
68082
names: ['robbyrussell/oh-my-zsh', 'github/gitignore', 'facebook/react-native', 'jwasham/coding-interview-university', 'angular/angular.js', 'electron/electron', 'torvalds/linux', 'FortAwesome/Font-Awesome', 'daneden/animate.css', 'jquery/jquery', 'freeCodeCamp/freeCodeCamp', 'twbs/bootstrap', 'EbookFoundation/free-programming-books', 'tensorflow/tensorflow', 'facebook/react', 'vuejs/vue', 'sindresorhus/awesome', 'getify/You-Dont-Know-JS', 'd3/d3', 'airbnb/javascript'] 

URLs: ['https://g

In [28]:
print('names:',len(name_list),'\n')
print('URLs:',len(url_list),'\n')
print('cont_list:',len(cont_list),'\n')
print('lang_list:',len(lang_list),'\n')
print('# of stars:',len(stars_list),'\n')
print('# of issues:',len(issues_list),'\n')
print('# of forks:',len(forks_list),'\n')

names: 20 

URLs: 20 

cont_list: 20 

lang_list: 20 

# of stars: 20 

# of issues: 20 

# of forks: 20 



In [63]:
# Convert list into DataFrame
reposDF = pd.DataFrame({"Repository Names":name_list,
                       "URLs":url_list,
                       "Contributors":cont_list,
                       "Language":lang_list,
                       "Stars":stars_list,
                       "Issues":issues_list,
                       "Forks":forks_list,
                       "ReadMe Length":rmlength_list})

reposDF


,Contributors,Forks,Issues,Language,ReadMe Length,Repository Names,Stars,URLs
0,"1,090","14,105",533,Shell,7298,robbyrussell/oh-my-zsh,"66,254",https://github.com/robbyrussell/oh-my-zsh
1,919,"28,052",169,,3268,github/gitignore,"61,867",https://github.com/github/gitignore
2,"1,609","13,832",563,JavaScript,4030,facebook/react-native,"60,193",https://github.com/facebook/react-native
3,104,"15,717",26,,66821,jwasham/coding-interview-university,"58,105",https://github.com/jwasham/coding-interview-un...
4,"1,604","28,805",503,JavaScript,5194,angular/angular.js,"58,049",https://github.com/angular/angular.js
5,732,"7,427",843,C++,2744,electron/electron,"56,961",https://github.com/electron/electron
6,∞,"20,509",195,C,720,torvalds/linux,"55,621",https://github.com/torvalds/linux
7,2,"9,530","4,387",CSS,3595,FortAwesome/Font-Awesome,"55,043",https://github.com/FortAwesome/Font-Awesome
8,84,"10,672",7,CSS,6045,daneden/animate.css,"49,366",https://github.com/daneden/animate.css
9,272,"14,427",71,JavaScript,10896,jquery/jquery,"48,115",https://github.com/jquery/jquery


In [64]:
# Save dataframe to file project_info.csv
reposDF.to_csv('project_info.csv', encoding='utf-8')

### 2. Use linear regression to analyze the Github repository data

In this part, you will analyze the data collectd in Part 1 using regression tools. The goal is to identify properties that make a repository popular. 

First, load the project_info.csv file in again. **We need you to do this so that we can run your code below withouth having to run your scraping code.**

In [ ]:
# this loads the data from the project_info.csv file 
project_info = pd.read_csv('project_info.csv')
project_info.head()

### 2.1. Reformat the data into useful datatypes

1. Print the data types for the columns of your dataframe
+ Reformat the colums Issues, Forks, Stars, and Contributers to be integer data
+ There is one repository flagged as having infinite contributers (the Linux kernel). We'll assume that it in fact has 15000 contributors (about twice as much as the next project). 

In [ ]:
# your code goes here


### Task 2.2 Describe the data

+ Get an overview of the data using the describe function.
+ Compute the correlation matrix, visualize it with a heat map.
+ Visualize the correlations by making a scatterplot matrix.
+ Interprete what you see.

You can re-use code from your previous homework here.

In [ ]:
# your code goes here

**Your Interpretation:** TODO

### 2.3. Linear regression

1. First use linear regression to try to predict the number of Stars based on Forks, Contributors, Issues, and Readme Length. Explain why this is not a very good model by discussing the R-squared , F-statistic p-value, and coefficient  p-values. 
+ Develop another model which is better. Explain why it is better and interpret your results. 

In [ ]:
# your code goes here

** Your interpretation:** TODO